In [1]:
from shared.simulators import ConstantEffectSimulator, ModeratedEffectSimulator, DataSimulator
import matplotlib.pyplot as plt

In [2]:
N = 50
T = 100

no_eff = ConstantEffectSimulator("No Effect", N, 0., 0., 0.).sample_params()
shared_random_eff = ConstantEffectSimulator("Shared Random Effect", N, 0., 1., 0.).sample_params()
distinct_random_eff = ConstantEffectSimulator("Distinct Random Effects", N, 0., 1., 1.).sample_params()
noisy_distinct_random_eff = ConstantEffectSimulator("Distinct Random Effects", N, 0., 1., 1., epsilon_sd=1.).sample_params()
moderated_eff = ModeratedEffectSimulator("Moderated Effects", N, 0., 1., 1., 0.5, 0.2, epsilon_sd=1.)
effect_simulators = [no_eff, shared_random_eff, distinct_random_eff, noisy_distinct_random_eff, moderated_eff]

for effect_simulator in effect_simulators:
    effect_simulator.sample_params()
    data_simulator = DataSimulator(N, T).simulate(effect_simulator)